<a href="https://colab.research.google.com/github/Daria-Lytvynenko/ML_course/blob/main/HW_2_4_%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%B8_%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%D1%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [24]:
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, auc, classification_report
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import lightgbm as lgb

raw_df=pd.read_csv('drive/MyDrive/ML_course/train.csv')
raw_df.drop(['Surname', 'CustomerId'], axis=1, inplace=True)

target_col='Exited'
input_cols=raw_df.iloc[:,:-1].columns.tolist()


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
train_df, val_df=split_train_val(raw_df, target_col)

In [26]:
train_inputs, train_targets= separate_inputs_targets(train_df, input_cols, target_col)

In [27]:
val_inputs, val_targets=separate_inputs_targets(val_df, input_cols, target_col)

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [28]:
train_inputs.head()

,id,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7180,7180,682.0,France,Male,30.0,1.0,131394.56,1.0,1.0,1.0,143952.24
10393,10393,684.0,France,Female,39.0,2.0,178058.06,1.0,1.0,0.0,145518.31
80,80,705.0,Germany,Male,35.0,6.0,116320.68,2.0,1.0,0.0,174431.01
3365,3365,669.0,Spain,Male,58.0,0.0,0.00,2.0,0.0,1.0,51565.98
12236,12236,707.0,France,Male,21.0,3.0,0.00,2.0,1.0,1.0,148564.76


In [29]:
cat_features=['Geography', 'Gender','HasCrCard', 'IsActiveMember']

In [30]:
train_inputs[cat_features]=train_inputs[cat_features].astype('category')
val_inputs[cat_features]=val_inputs[cat_features].astype('category')

2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [31]:
xgb_clf = XGBClassifier(
    n_estimators=50,
    max_depth=5,
    enable_categorical=True,
    missing=np.nan,
    device='cuda',
    alpha=50,
    scale_pos_weight=4
)

xgb_clf.fit(train_inputs, train_targets)
pred_train=xgb_clf.predict(train_inputs)
pred_val=xgb_clf.predict(val_inputs)
pred_proba_train=xgb_clf.predict_proba(train_inputs)[:,1]
pred_proba_val=xgb_clf.predict_proba(val_inputs)[:,1]
fpr_tr, tpr_tr, threshold_tr=roc_curve(train_targets, pred_proba_train)
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
roc_auc_tr=auc(fpr_tr, tpr_tr)
roc_auc_val=auc(fpr_val, tpr_val)
print(classification_report(train_targets, pred_train, digits=4))
print(classification_report(val_targets, pred_val, digits=4))

              precision    recall  f1-score   support

         0.0     0.9614    0.8739    0.9156      9558
         1.0     0.6362    0.8628    0.7324      2442

    accuracy                         0.8717     12000
   macro avg     0.7988    0.8684    0.8240     12000
weighted avg     0.8952    0.8717    0.8783     12000

              precision    recall  f1-score   support

         0.0     0.9594    0.8607    0.9074      2390
         1.0     0.6110    0.8574    0.7135       610

    accuracy                         0.8600      3000
   macro avg     0.7852    0.8590    0.8104      3000
weighted avg     0.8886    0.8600    0.8679      3000



In [32]:
roc_auc_tr, roc_auc_val

(0.9431224796102385, 0.9342893888469717)

Модель на мою думку хороша, якість трохи покращилась порівняно з DecisionTrees. Перенавчання чи недонавчання не спостерігається.

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [33]:
!pip install hyperopt

In [34]:
def objective(params):
    clf = xgb.XGBClassifier(
    n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        scale_pos_weight=params['scale_pos_weight'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        enable_categorical=True,
        missing=np.nan,
        device='cuda'
    )

    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)],
        verbose=False)
    pred = clf.predict(val_inputs)
    pred_proba_val=clf.predict_proba(val_inputs)[:,1]
    fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba_val)
    roc_auc_val=auc(fpr_val, tpr_val)

    return {'loss': -roc_auc_val, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 10, 1000, 5),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
    'subsample': hp.uniform('subsample', 0.3, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'scale_pos_weight':hp.uniform('scale_pos_weight', 0.1, 10)
}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])

print("Найкращі гіперпараметри: ", best)

final_clf = xgb.XGBClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    gamma=best['gamma'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    enable_categorical=True,
    missing=np.nan,
    device='cuda',
    scale_pos_weigth=best['scale_pos_weight']
)

final_clf.fit(train_inputs, train_targets)
final_pred = final_clf.predict(val_inputs)
final_pred_proba=final_clf.predict_proba(val_inputs)[:,1]
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, final_pred_proba)
final_roc_auc=auc(fpr_val, tpr_val)
print("Roc auc на валідаційній вибірці: {:.4f}".format(final_roc_auc))
print(classification_report(val_targets, final_pred, digits=4))

100%|██████████| 20/20 [00:41<00:00,  2.08s/trial, best loss: -0.9326689073324647]
Найкращі гіперпараметри:  {'colsample_bytree': 0.8642488240844555, 'gamma': 0.1519737464238179, 'learning_rate': 0.03202434674949165, 'max_depth': 6, 'min_child_weight': 13, 'n_estimators': 195, 'reg_alpha': 0.8482468248446372, 'reg_lambda': 0.8678732103596338, 'scale_pos_weight': 4.245661922593994, 'subsample': 0.43186026085483264}
Roc auc на валідаційній вибірці: 0.9338
              precision    recall  f1-score   support

         0.0     0.9211    0.9527    0.9367      2390
         1.0     0.7860    0.6803    0.7293       610

    accuracy                         0.8973      3000
   macro avg     0.8536    0.8165    0.8330      3000
weighted avg     0.8936    0.8973    0.8945      3000



Я вважаю, що дана модель краща за попередню, оскільки згідно звіту оцінка f1 для обох класів вища то точність визначення цільового класу (Exited=1) в даному випадку вища.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [35]:
!pip install dask[dataframe]

In [36]:
cat_num=[train_inputs.columns.get_loc(col) for col in cat_features]

In [37]:
lgb_clf = lgb.LGBMClassifier(
    max_depth=5,
    n_estimators=100,
    learning_rate=0.1,
    scale_pos_weight=4,
    cat_feature=cat_num,
    missing=np.nan
)

lgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])

train_pred = lgb_clf.predict(train_inputs)
val_pred = lgb_clf.predict(val_inputs)
pred_proba=lgb_clf.predict_proba(val_inputs)[:,1]
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba)
roc_auc=auc(fpr_val, tpr_val)
print(roc_auc)
print(classification_report(train_targets, train_pred, digits=4))
print(classification_report(val_targets, val_pred, digits=4))

[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] categorical_feature is set with cat_feature=2,3,8,9, categorical_column=2,3,8,9 will be ignored. Current value: categorical_feature=2,3,8,9
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 11
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Якість моделі не покращилась.

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [38]:
warnings.filterwarnings("ignore")

def objective(params):
    clf = lgb.LGBMClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        num_leaves=int(params['num_leaves']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        min_split_gain=params['min_split_gain'],
        cat_feature=cat_num,
        scale_pos_weight=params['scale_pos_weight']
    )

    clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])
    pred = clf.predict(val_inputs)
    pred_proba=clf.predict_proba(val_inputs)[:,1]
    fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba)
    roc_auc=auc(fpr_val, tpr_val)

    return {'loss': -roc_auc, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 200, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.1),
    'scale_pos_weight':hp.uniform('scale_pos_weight', 0.1, 10)}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_weight'] = int(best['min_child_weight'])

print("Найкращі гіперпараметри: ", best)

final_lgb_clf = lgb.LGBMClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    num_leaves=best['num_leaves'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    min_split_gain=best['min_split_gain'],
    cat_feature=cat_num,
    missing=np.nan,
    scale_pos_weight=best['scale_pos_weight']
)

final_lgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])
final_pred = final_lgb_clf.predict(val_inputs)
pred_proba=final_lgb_clf.predict_proba(val_inputs)[:,1]
fpr_val, tpr_val, threshold_val=roc_curve(val_targets, pred_proba)
roc_auc_final=auc(fpr_val, tpr_val)

print("Точність на валідаційній вибірці: {:.4f}".format(roc_auc_final))

print(classification_report(val_targets, final_pred, digits=4))

[LightGBM] [Warning] categorical_feature is set with cat_feature=2,3,8,9, categorical_column=2,3,8,9 will be ignored. Current value: categorical_feature=2,3,8,9
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001126 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

Дана модель має показники, подібні до моделі з п.3

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

Я обрала останню модель, оскільки вона має вище значення roc auc порівняно з іншими моделями та точніше і повніше визначає клієнтів, які перестануть користуватись послугами банку.

In [39]:
test=pd.read_csv('drive/MyDrive/ML_course/test.csv')
test.drop(['Surname', 'CustomerId'], axis=1, inplace=True)

In [40]:
sample_submission=pd.read_csv('drive/MyDrive/ML_course/sample_submission.csv')

In [41]:
test[cat_features]=test[cat_features].astype('category')

In [42]:
train_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 7180 to 9360
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   id               12000 non-null  int64   
 1   CreditScore      12000 non-null  float64 
 2   Geography        12000 non-null  category
 3   Gender           12000 non-null  category
 4   Age              12000 non-null  float64 
 5   Tenure           12000 non-null  float64 
 6   Balance          12000 non-null  float64 
 7   NumOfProducts    12000 non-null  float64 
 8   HasCrCard        12000 non-null  category
 9   IsActiveMember   12000 non-null  category
 10  EstimatedSalary  12000 non-null  float64 
dtypes: category(4), float64(6), int64(1)
memory usage: 797.4 KB


In [43]:
sample_submission['Exited']=final_lgb_clf.predict(test)

In [44]:
sample_submission.set_index('id')

,Exited
id,
15000,0
15001,0
15002,0
15003,0
15004,0
...,...
24995,0
24996,0
24997,0


In [45]:
sample_submission.to_csv('drive/MyDrive/ML_course/submission_lgbm.csv', index=False)